<h1>Casestudy Eric Bühler - Aachen </h1>

In this notebook, some initial data analysis along with the regressions are calculated.

In [4]:
# Packages used in this notebook
from nltk.sentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

<h3>Analysis of Verbindung and Verbindung möglich variables</h3>

To avoid confusion with the fraternity variable, I have included two ways of evaluating wether a listing is a fraternity: Either by checking the provided texts for synonyms of fraternity or, since not all fraternities identify themselves as such, by looking for listings with low rent and a high number of roomates. To avoid colinearity however, only one of these should be included in the regression. To check wether we are not loosing a big amount of information, the following code checks the overlap of the two variables.

In [7]:
# Load the CSV file into a DataFrame
file_path = '/workspaces/fdap-2024-Big-Eric-Blip/casestudy/student_housing/data_analysis/anzeigen.csv'  # Path to the CSV file
anzeigen = pd.read_csv(file_path)

# Condition checks
condition_one = (anzeigen['verbindung'] == True) & (anzeigen['verbindung_moeglich'] == False)
condition_two = (anzeigen['verbindung'] == False) & (anzeigen['verbindung_moeglich'] == True)

# Check condition one
rows_with_condition_one = anzeigen[condition_one]
if not rows_with_condition_one.empty:
    print("There is at least one row where 'verbindung' is True and 'verbindung_moeglich' is False.")
else:
    print("There are no rows where 'verbindung' is True and 'verbindung_moeglich' is False.")

num_rows_one = condition_one.sum()
print(f"There are {num_rows_one} rows where 'verbindung' is True and 'verbindung_moeglich' is False.")

# Check condition two
rows_with_condition_two = anzeigen[condition_two]
if not rows_with_condition_two.empty:
    print("There is at least one row where 'verbindung' is False and 'verbindung_moeglich' is True.")
else:
    print("There are no rows where 'verbindung' is False and 'verbindung_moeglich' is True.")

num_rows_two = condition_two.sum()
print(f"There are {num_rows_two} rows where 'verbindung' is False and 'verbindung_moeglich' is True.")


There is at least one row where 'verbindung' is True and 'verbindung_moeglich' is False.
There are 2 rows where 'verbindung' is True and 'verbindung_moeglich' is False.
There is at least one row where 'verbindung' is False and 'verbindung_moeglich' is True.
There are 19 rows where 'verbindung' is False and 'verbindung_moeglich' is True.


<h2>Regressions</h2>

<h3>Analysis of rent</h3>

In the following code, we calculate a regression on the dependent variable 'price' by the independant variables bewohner, goresse, miete and plz.

In [9]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Define the necessary columns for the DataFrame
necessary_columns = ['bewohner', 'groesse', 'miete', 'plz']

# Load the CSV file into a DataFrame
file_path = '/workspaces/fdap-2024-Big-Eric-Blip/casestudy/student_housing/data_analysis/anzeigen.csv'  # Path to the CSV file
df = pd.read_csv(file_path)

# Create a DataFrame with only the necessary columns
df = df[necessary_columns]

# Convert numeric columns to numeric types and handle errors
df['bewohner'] = pd.to_numeric(df['bewohner'], errors='coerce')
df['groesse'] = pd.to_numeric(df['groesse'], errors='coerce')
df['miete'] = pd.to_numeric(df['miete'], errors='coerce')

# To ensure the rent is in rent per square meter:
df['miete'] = df['miete'] / df['groesse']

# Drop rows with missing values in numeric columns
df = df.dropna(subset=['bewohner', 'miete'])

# Convert 'plz' to categorical and create dummy variables
df['plz'] = df['plz'].astype('category')
df = pd.get_dummies(df, columns=['plz'], drop_first=True)

# Add the intercept (constant term) for the regression model
df = sm.add_constant(df)

# Round numeric values to a specified number of decimal places
df = df.round({'bewohner': 0, 'miete': 2})

# Prepare the data for regression
X = df.drop(columns=['miete', 'groesse'])  # Exclude dependent variable 'miete'
y = df['miete']

# Ensure all columns used for regression are numeric
X = X.apply(pd.to_numeric, errors='coerce').fillna(0)

# Convert boolean columns to int64
bool_columns = X.select_dtypes(include='bool').columns
X[bool_columns] = X[bool_columns].astype('int64')

# Fit the model using the original DataFrame to retain column names
model = sm.OLS(y, X).fit()

# Print the summary of the regression with custom variable names
print("Regression Summary:")
print(model.summary())  # Coefficients table

# Extract regression results
results_df = pd.DataFrame({
    "Coefficients": model.params,
    "P-Values": model.pvalues,
    "Standard Errors": model.bse,
    "t-values": model.tvalues,
    "Confidence Interval Lower": model.conf_int()[0],
    "Confidence Interval Upper": model.conf_int()[1]
})

# Save results to CSV
results_df.to_csv('/workspaces/fdap-2024-Big-Eric-Blip/casestudy/student_housing/results/rent_regression_results.csv', index=True)


Regression Summary:
                            OLS Regression Results                            
Dep. Variable:                  miete   R-squared:                       0.242
Model:                            OLS   Adj. R-squared:                  0.206
Method:                 Least Squares   F-statistic:                     6.700
Date:                Tue, 09 Jul 2024   Prob (F-statistic):           9.98e-10
Time:                        17:48:44   Log-Likelihood:                -830.21
No. Observations:                 243   AIC:                             1684.
Df Residuals:                     231   BIC:                             1726.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         32.1251      1.267


<h3>Analysis of listing-duration</h3>

In the following code, we calculate a regression on the dependent variable 'online_seit' by the independant variables bewohner, goresse, miete, sentiment, verbindung and plz.

In [36]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Define the necessary columns for the DataFrame
necessary_columns = ['bewohner', 'groesse', 'miete', 'plz', 'online_seit', 'sentiment', 'verbindung']

# Load the CSV file into a DataFrame
file_path = '/workspaces/fdap-2024-Big-Eric-Blip/casestudy/student_housing/data_analysis/anzeigen.csv'  # Path to the CSV file
df = pd.read_csv(file_path)

# Create a DataFrame with only the necessary columns
df = df[necessary_columns]

# Convert numeric columns to numeric types and handle errors
df['bewohner'] = pd.to_numeric(df['bewohner'], errors='coerce')
df['groesse'] = pd.to_numeric(df['groesse'], errors='coerce')
df['miete'] = pd.to_numeric(df['miete'], errors='coerce')
df['sentiment'] = pd.to_numeric(df['sentiment'], errors='coerce') * 100
df['online_seit'] = pd.to_numeric(df['online_seit'], errors='coerce')
df['sentiment'] = pd.to_numeric(df['sentiment'], errors='coerce')
df['verbindung'] = pd.to_numeric(df['verbindung'], errors='coerce')

# To ensure the rent is in rent per square meter:
df['miete'] = df['miete'] / df['groesse']

# Drop rows with missing values in numeric columns
df = df.dropna(subset=['bewohner', 'groesse', 'miete'])

# Convert 'plz' and 'verbindung' to categorical and create dummy variables
df['plz'] = df['plz'].astype('category')
df['verbindung'] = df['verbindung'].astype('category')
df = pd.get_dummies(df, columns=['plz'], drop_first=True)
df = pd.get_dummies(df, columns=['verbindung'], drop_first=True)

# Add the intercept (constant term) for the regression model
df = sm.add_constant(df)

# Round numeric values to a specified number of decimal places
df = df.round({'bewohner': 0, 'groesse': 0, 'miete': 2, 'sentiment': 5})

# Prepare the data for regression
X = df.drop(columns=['online_seit'])  # Exclude dependent variable 'online_seit'
y = df['online_seit']

# Ensure all columns used for regression are numeric
X = X.apply(pd.to_numeric, errors='coerce')  # Convert to numeric, forcing non-numeric to NaN
X = X.fillna(0)  # Fill NaNs with 0 or some other value depending on context

# Convert boolean columns to int64
bool_columns = X.select_dtypes(include='bool').columns
X[bool_columns] = X[bool_columns].astype('int64')

# Fit the model using the original DataFrame to retain column names
model = sm.OLS(y, X).fit()

# Print the summary of the regression with custom variable names
print("Regression Summary:")
print(model.summary())  # Coefficients table

# Extract regression results
results_df = pd.DataFrame({
    "Coefficients": model.params,
    "P-Values": model.pvalues,
    "Standard Errors": model.bse,
    "t-values": model.tvalues,
    "Confidence Interval Lower": model.conf_int()[0],
    "Confidence Interval Upper": model.conf_int()[1]
})


# Save results to CSV
results_df.to_csv('/workspaces/fdap-2024-Big-Eric-Blip/casestudy/student_housing/results/listing_duration_regression_results.csv', index=True)


Regression Summary:
                            OLS Regression Results                            
Dep. Variable:            online_seit   R-squared:                       0.115
Model:                            OLS   Adj. R-squared:                  0.057
Method:                 Least Squares   F-statistic:                     1.975
Date:                Tue, 09 Jul 2024   Prob (F-statistic):             0.0180
Time:                        18:25:57   Log-Likelihood:                -1585.5
No. Observations:                 243   AIC:                             3203.
Df Residuals:                     227   BIC:                             3259.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const             123.

<h3>Evaluating NDVI of the districts</h3>

In this cell, we extract the NDVI per PLZ from the [urban_green_spaces.ipynb](/workspaces/fdap-2024-Big-Eric-Blip/casestudy/student_housing/google_earth_engine/urban_green_spaces.ipynb) notebook and order them by descending NDVI. Also, we added the effect the plz had on price and listing duration, so we can evaluate wether the effect of ugs suggested by the papers can be confirmed.

In [19]:
import pandas as pd

# Load the CSV data into DataFrames
ndvi_df = pd.read_csv('/workspaces/fdap-2024-Big-Eric-Blip/casestudy/student_housing/google_earth_engine/ndvi_2020_results.csv')
anzeigen_df = pd.read_csv('/workspaces/fdap-2024-Big-Eric-Blip/casestudy/student_housing/data_analysis/anzeigen.csv')
rent_df = pd.read_csv('/workspaces/fdap-2024-Big-Eric-Blip/casestudy/student_housing/results/rent_regression_results.csv')
duration_df = pd.read_csv('/workspaces/fdap-2024-Big-Eric-Blip/casestudy/student_housing/results/listing_duration_regression_results.csv')

# Some data alteration to make the DFs compatible
#Rename Columns
rent_df.columns.values[0] = 'Variables'
rent_df.columns.values[1] = 'Rent Coefficients'
duration_df.columns.values[0] = 'Variables'
duration_df.columns.values[1] = 'Duration Coefficients'

# Drop unneeded columns and rows
rent_df = rent_df.drop(columns=['P-Values','Standard Errors','t-values','Confidence Interval Lower','Confidence Interval Upper'])
rent_df = rent_df.iloc[2:]
rent_df['Variables'] = rent_df['Variables'].str.replace('plz_', '')

duration_df = duration_df.drop(columns=['P-Values','Standard Errors','t-values','Confidence Interval Lower','Confidence Interval Upper'])
duration_df = duration_df.iloc[5:-1]
duration_df['Variables'] = duration_df['Variables'].str.replace('plz_', '')

# Get the PLZ Counts
plz_counts = anzeigen_df['plz'].value_counts().reset_index()
plz_counts.columns = ['plz', 'count']

# Merge the count data with the sorted NDVI DataFrame
sorted_ndvi_df = ndvi_df.sort_values(by='mean_ndvi', ascending=False)
merged_df = sorted_ndvi_df.merge(plz_counts, left_on='plz_code', right_on='plz', how='left')

# Drop the extra 'plz' column from the right DataFrame after merge
merged_df = merged_df.drop(columns=['plz'])

# Fill NaN values with 0 (if any plz in ndvi_df is not present in anzeigen_df)
merged_df['count'] = merged_df['count'].fillna(0).astype(int)

# Convert 'plz_code' in merged_df to string
merged_df['plz_code'] = merged_df['plz_code'].astype(str)

# Merge with rent_df and duration_df
merged_df = merged_df.merge(rent_df, left_on='plz_code', right_on='Variables', how='left')
merged_df = merged_df.merge(duration_df, left_on='plz_code', right_on='Variables', how='left')

# Drop the extra Columns
merged_df = merged_df.drop(columns=['Variables_x','Variables_y'])

# Display the merged DataFrame
display(merged_df)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('merged_ndvi_2020_results.csv', index=False)


,plz_code,mean_ndvi,count,Rent Coefficients,Duration Coefficients
0,52076,0.283134,0,NaN,NaN
1,52074,0.282947,25,0.948012,-89.584758
2,52080,0.239880,2,-0.625412,-103.151223
3,52078,0.230341,6,-3.189890,84.408749
4,52072,0.229945,33,1.162275,-21.205014
5,52066,0.207245,24,-0.841410,-64.529513
6,52070,0.200926,42,-1.873405,-70.107180
7,52068,0.145891,9,-0.662744,-56.394690
8,52064,0.135038,50,-1.500974,-47.213212
9,52062,0.099158,48,NaN,NaN
